In [1]:
import pandas as pd
import numpy as np
import os
import sys

file_directory=os.getcwd()+'/'
file_name1='train set-multiscale.csv'
data_train=pd.read_csv(file_directory+file_name1,engine='python')
file_name2='test set-multiscale.csv'
data_test=pd.read_csv(file_directory+file_name2,engine='python')

In [2]:
X_train=data_train.iloc[:,2:52]
Y_train=data_train.iloc[:,0]
X_test=data_test.iloc[:,2:52]
Y_test=data_test.iloc[:,0]

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号
%matplotlib inline
sns.set_style("whitegrid")
sns.set(rc = {'figure.figsize':(100, 100)})
def imp_df(column_names, importances):
    df = pd.DataFrame({'feature': column_names,
                       'feature_importance': importances}) \
           .sort_values('feature_importance', ascending = False) \
           .reset_index(drop = True)
    return df

# plotting a feature importance dataframe (horizontal barchart)
def var_imp_plot(imp_df, title):
    imp_df.columns = ['feature', 'feature_importance']
    sns.barplot(x = 'feature_importance', y = 'feature', data = imp_df, orient = 'h', color = 'royalblue') \
       .set_title(title, fontsize = 12)
    

In [4]:
def IncMSE(MSE,x_test, y_test,FeatureNum,Set_Times,model,VI):    #获取MSE，x测试集，y测试集，特征数，随机求IncMSE次数，模型（随机森林）
    x_MSE = copy.deepcopy(x_test)      #深拷贝不破坏原列表
    y_MSE = copy.deepcopy(y_test)
    TestNum = len(y_MSE)
    #########多次生成随机数，多次计算IncMSE（由于随机有不确定性，所以要多次随机）
    IncMSE_Set = []
    IncMSE_Times = 1
    while IncMSE_Times <= Set_Times:     #多次生成随机数，多次计算IncMSE（由于随机有不确定性，所以要多次随机）
        IncMSE_x = []
        for i in range(0,FeatureNum):
            MSE_Replace = np.random.random(TestNum)
            x_MSE[:,i] = MSE_Replace           #替换第i个特征
            MSE_Score = model.score(x_MSE,y_MSE)
            MSE_Result = model.predict(x_MSE)
            MSE_ResidualSquare = (MSE_Result - y_MSE)**2   #计算残差平方
            MSE_RSS = sum(MSE_ResidualSquare)   #计算残差平方和
            MSE_MSE = np.mean(MSE_ResidualSquare)   #计算均方差
            IncMSE = MSE_MSE - MSE
            IncMSE_x.append(IncMSE)
            x_MSE = copy.deepcopy(x_test)   #复原原特征，深拷贝不破坏原列表            
        IncMSE_Set.append(IncMSE_x)          #多次计算IncMSE后的数据
        
        IncMSE_Times += 1
    IncMSE_SetArray = np.array(IncMSE_Set)    #变换为array
    ########计算每个特征的IncMSE平均数########
    X_IncMSE_Average = []
    for j in range(0,FeatureNum):
        X_IncMSE_Set = IncMSE_SetArray[:,j]
        X_IncMSE = np.mean(X_IncMSE_Set)       #求多次IncMSE的平均值（由于随机有不确定性，所以要多次随机）
        X_IncMSE_Average.append(X_IncMSE)
        VI.append(X_IncMSE)
    X_IncMSE_Average_Sum = sum(X_IncMSE_Average)
    ########计算每个特征的IncMSE平均数的百分比########
    print('IncMSE:')
    for k in range(0,FeatureNum):
        X_Percent = X_IncMSE_Average[k]/X_IncMSE_Average_Sum       #计算每个特征IncMSE的百分比
        print(f'    x{k+1} = {X_IncMSE_Average[k]}   {X_Percent*100}%')        #输出各特征的IncMSE的平均数与其百分比

In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn import ensemble
import copy


D:\anaconda\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [6]:
model = RandomForestRegressor(n_estimators = 200,
                               n_jobs = -1,max_features=8,max_depth=10,
                               oob_score = True,
                               bootstrap = True,
                               random_state = 42)
model.fit(X_train, Y_train)

score = model.score(X_test, Y_test)
result = model.predict(X_test)
ResidualSquare = (result - Y_test)**2     #计算残差平方
RSS = sum(ResidualSquare)   #计算残差平方和
MSE = np.mean(ResidualSquare)      #计算均方差
RMSE = np.mean(ResidualSquare)**0.5      #计算均方差
num_regress = len(result)   #回归样本个数


VI=[]

x_valid2=X_test.values
y_valid2=Y_test.values
IncMSE(MSE,x_valid2,y_valid2,50,20,model,VI)

IncMSE:
    x1 = -0.008544689829620266   -0.5532707646605749%
    x2 = -0.034393363394763274   -2.2269787252787103%
    x3 = 0.013509093769764136   0.8747171388198851%
    x4 = -0.021956830486547417   -1.4217101656169284%
    x5 = 0.003294953229856068   0.21334903072594194%
    x6 = 0.02185616636153137   1.4151921388035442%
    x7 = 0.06513173156231661   4.217295612092646%
    x8 = 0.03263905635151223   2.113386913449062%
    x9 = 0.04255378771085452   2.755368203579902%
    x10 = 0.09162219749686172   5.932559786224995%
    x11 = 0.007521551747395916   0.48702232259971456%
    x12 = 0.031189749593631376   2.0195439449915265%
    x13 = 0.0557060200617416   3.606978477898336%
    x14 = 0.004703481017269795   0.30455155083224433%
    x15 = 0.005970790123746461   0.3866101266708966%
    x16 = 0.05282237173744707   3.420261540805545%
    x17 = 0.01841047135406384   1.192082540961839%
    x18 = 0.00010335964237678796   0.006692562224390353%
    x19 = 0.002851197636856484   0.184615747112744

In [7]:
file_directory=file_directory=os.getcwd()+'/'
file_name1='train set-optimal scale.csv'
data_train=pd.read_csv(file_directory+file_name1,engine='python')
file_name2='test set-optimal scale.csv'
data_test=pd.read_csv(file_directory+file_name2,engine='python')
file_name3='validation set-optimal scale.csv'
data_valid=pd.read_csv(file_directory+file_name3,engine='python')
data_train.describe()

,Y,age35_twogrid,local_threegrid,people_threegrid,rich1_threegrid,路网可达性,网吧j1500,娱乐场所j1000,住宅区j1000,atmj1500,医院j1500,non_local_twogrid,age25_threegrid
count,5540.000000,5540.000000,5540.000000,5540.000000,5540.000000,5540.000000,5540.000000,5540.000000,5540.000000,5540.000000,5540.000000,5540.000000,5540.000000
mean,1.340195,0.168279,0.445181,16918.912274,0.000034,31.355041,6.362816,11.101083,53.851083,39.151264,19.590614,0.511357,0.285504
std,2.990523,0.068164,0.179573,22730.450618,0.000123,13.385704,10.644010,21.973557,86.591956,72.484831,40.963049,0.185766,0.092502
min,0.000000,0.000000,0.000000,3.000000,0.000000,2.800081,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.143294,0.313309,1185.750000,0.000000,20.496530,0.000000,0.000000,2.000000,0.000000,0.000000,0.366383,0.248207
50%,0.000505,0.171445,0.441925,4884.000000,0.000000,30.538046,1.000000,0.000000,9.000000,5.000000,2.000000,0.506084,0.287850
75%,1.180362,0.193425,0.577606,25195.000000,0.000000,40.453534,9.000000,11.000000,68.000000,39.000000,16.000000,0.647510,0.327104
max,24.107420,1.166667,1.000000,271068.000000,0.001530,96.117348,56.000000,126.000000,401.000000,424.000000,221.000000,1.000000,1.000000


In [8]:
X_train=data_train.iloc[:,1:13]
Y_train=data_train.iloc[:,0]
X_test=data_test.iloc[:,1:13]
Y_test=data_test.iloc[:,0]
X_valid=data_valid.iloc[:,1:13]
Y_valid=data_valid.iloc[:,0]
from sklearn.ensemble import RandomForestRegressor
from sklearn import ensemble

In [9]:
model = RandomForestRegressor(n_estimators = 40,
                               max_features=4,max_depth=10, random_state=42, bootstrap=True,  n_jobs=-1)
model.fit(X_train, Y_train)

score = model.score(X_test, Y_test)
result = model.predict(X_test)
ResidualSquare = (result - Y_test)**2     #计算残差平方
RSS = sum(ResidualSquare)   #计算残差平方和
MSE = np.mean(ResidualSquare)      #计算均方差
RMSE = np.mean(ResidualSquare)**0.5      #计算均方差
num_regress = len(result)   #回归样本个数
print(f'n={num_regress}')
print(f'R^2={score}')
print(f'RMSE={RMSE}')
print(f'RSS={RSS}')

n=2750
R^2=0.8254369670576089
RMSE=1.1549759115044664
RSS=3668.415729427826


In [10]:
base_imp = imp_df(X_train.columns, model.feature_importances_)
base_imp

,feature,feature_importance
0,网吧j1500,0.278086
1,娱乐场所j1000,0.171796
2,people_threegrid,0.147511
3,住宅区j1000,0.106260
4,rich1_threegrid,0.080599
5,路网可达性,0.060348
6,医院j1500,0.048776
7,atmj1500,0.033027
8,non_local_twogrid,0.026960
9,local_threegrid,0.023491
